In [1]:
# Import Dependencies
import pandas as pd
import sqlite3 as sql
import os

In [2]:
## Based on an example found at https://www.kaggle.com/code/agilesifaka/historic-nba-drafting-game-and-player-analysis

# Connect to SQL database
db_path = 'Basketball_Dataset/basketball.sqlite'
connection = sql.connect(db_path) # create connection object to database
print("SQL database connected")
table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", connection)
print(table)

SQL database connected
                     name
0                  Player
1                    Team
2         Team_Attributes
3            Team_History
4       Player_Attributes
5          Game_Officials
6   Game_Inactive_Players
7             Team_Salary
8           Player_Salary
9                   Draft
10          Draft_Combine
11          Player_Photos
12            Player_Bios
13                   Game
14                   News
15           News_Missing


In [3]:
# Query the Draft table
query = """
    SELECT 
        yearDraft,
        numberPickOverall,
        numberRound,
        namePlayer,
        idPlayer
    FROM Draft
    WHERE yearDraft >= 1980;
"""
basketball_db_Draft_df = pd.read_sql(query, connection)
basketball_db_Draft_df

,yearDraft,numberPickOverall,numberRound,namePlayer,idPlayer
0,2020.0,1.0,1.0,Anthony Edwards,1630162.0
1,2020.0,2.0,1.0,James Wiseman,1630164.0
2,2020.0,3.0,1.0,LaMelo Ball,1630163.0
3,2020.0,4.0,1.0,Patrick Williams,1630172.0
4,2020.0,5.0,1.0,Isaac Okoro,1630171.0
...,...,...,...,...,...
3533,1980.0,210.0,10.0,Melvin Crayton,82554.0
3534,1980.0,211.0,10.0,Randy Carroll,82563.0
3535,1980.0,212.0,10.0,Kent Williams,82552.0
3536,1980.0,213.0,10.0,Joe Hand,82493.0


In [4]:
# Force numeric values to integers

basketball_db_Draft_df['yearDraft'] = basketball_db_Draft_df['yearDraft'].astype(int)
basketball_db_Draft_df['numberPickOverall'] = basketball_db_Draft_df['numberPickOverall'].astype(int)
basketball_db_Draft_df['numberRound'] = basketball_db_Draft_df['numberRound'].astype(int)
basketball_db_Draft_df['idPlayer'] = basketball_db_Draft_df['idPlayer'].astype(int)

#basketball_db_Draft_df.dtypes

In [5]:
# Create Unique Identifier Field - Concatination of Name and Draft Year
# Converted to All Upper Case, All Punctuation and Spaces Stripped
basketball_db_Draft_df['GUID'] = basketball_db_Draft_df['namePlayer'] + basketball_db_Draft_df['yearDraft'].astype(str)

# Strip out Single Quote/Apostrophe Characters, Commas, and Periods
basketball_db_Draft_df['GUID'] = basketball_db_Draft_df['GUID'].str.replace(r'[^\w\s]+', '')

# Strip out spaces
basketball_db_Draft_df['GUID'].replace('\s+', '', regex=True,inplace=True)

#Convert to Upper Case
basketball_db_Draft_df['GUID'] = basketball_db_Draft_df['GUID'].str.upper()

basketball_db_Draft_df.head(100)

C:\Users\18177\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


,yearDraft,numberPickOverall,numberRound,namePlayer,idPlayer,GUID
0,2020,1,1,Anthony Edwards,1630162,ANTHONYEDWARDS2020
1,2020,2,1,James Wiseman,1630164,JAMESWISEMAN2020
2,2020,3,1,LaMelo Ball,1630163,LAMELOBALL2020
3,2020,4,1,Patrick Williams,1630172,PATRICKWILLIAMS2020
4,2020,5,1,Isaac Okoro,1630171,ISAACOKORO2020
...,...,...,...,...,...,...
95,2019,36,2,Cody Martin,1628998,CODYMARTIN2019
96,2019,37,2,Deividas Sirvydis,1629686,DEIVIDASSIRVYDIS2019
97,2019,38,2,Daniel Gafford,1629655,DANIELGAFFORD2019
98,2019,39,2,Alen Smailagic,1629346,ALENSMAILAGIC2019


In [6]:
# Set new index and drop existing index
basketball_db_Draft_df.set_index('GUID')

,yearDraft,numberPickOverall,numberRound,namePlayer,idPlayer
GUID,,,,,
ANTHONYEDWARDS2020,2020,1,1,Anthony Edwards,1630162
JAMESWISEMAN2020,2020,2,1,James Wiseman,1630164
LAMELOBALL2020,2020,3,1,LaMelo Ball,1630163
PATRICKWILLIAMS2020,2020,4,1,Patrick Williams,1630172
ISAACOKORO2020,2020,5,1,Isaac Okoro,1630171
...,...,...,...,...,...
MELVINCRAYTON1980,1980,210,10,Melvin Crayton,82554
RANDYCARROLL1980,1980,211,10,Randy Carroll,82563
KENTWILLIAMS1980,1980,212,10,Kent Williams,82552


In [7]:
# Check datatypes
basketball_db_Draft_df.dtypes

yearDraft             int32
numberPickOverall     int32
numberRound           int32
namePlayer           object
idPlayer              int32
GUID                 object
dtype: object

In [8]:
# Identify incomplete rows
basketball_db_Draft_df.count()

yearDraft            3538
numberPickOverall    3538
numberRound          3538
namePlayer           3538
idPlayer             3538
GUID                 3538
dtype: int64

In [9]:
# import the necessary packages
import psycopg2
from sqlalchemy import create_engine
  
# Create the engine to connect to the database 
engine = create_engine(
    '')

# Write data into the table in database
basketball_db_Draft_df.to_sql('basketball_db_Draft', engine)